In [ ]:
# Greg Wellman - February 2022
# ============================
from html.parser import HTMLParser

# String helpers
def left(s, amount):
    return s[:amount]
def right(s, amount):
    return s[-amount:]
def mid(s, offset, amount):
    return s[offset:offset+amount]

# Read file into string
html_file = open('cdp-2022-cc.html', 'r')
cdp_data = html_file.read()
html_file.close()

# Parse HTML
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.bInSection = False
        self.bParseForTable = False
        self.bAllowData = True
        self.iDataType = 0
        self.iCounter = 0
        self.sMaster = ''
        self.sSub = ''
        self.dictQ = {}

    def handle_starttag(self, tag, attrs):        
        if tag == 'section': self.bInSection = True
            
        if tag == 'div' and attrs[0] == ('class', 'content'):
            if attrs[1][1] == 'c0-introduction': self.bParseForTable = True    
            self.bAllowData = True
            
        if tag == 'h1':
            self.iDataType = 1
        elif tag == 'h2':
            self.iDataType = 2
        elif tag == 'h3':
            self.iDataType = 3
        else:
            self.iDataType = 0
                
    def handle_endtag(self, tag):        
        if tag == 'section': self.bInSection = False            
    
    def handle_data(self, data):        
        d = data.strip()
        if d == 'Response options': self.bAllowData = False
        if self.bInSection and self.bParseForTable and self.bAllowData and len(d) > 0:
            if self.iDataType == 1: 
                self.sMaster = d
                self.dictQ.update({'Master': d})
            elif self.iDataType == 2: 
                self.sSub = d                
                self.dictQ.update({'Sub': d})
            elif self.iDataType == 3: 
                
                # Check for question
                if left(d, 1) == '(':
                    print('Found open parenthesis')
                
            else:
                print(self.iCounter, self.sMaster, self.sSub, d)
                self.iCounter += 1
                
        print(self.dictQ)
        
parser = MyHTMLParser()
parser.feed(cdp_data)